# Photovoltaic Fleet ddDT Demo
This notebook is an example of training a ddDT on a fleet of PV systems.

Before running this notebook, make sure the package is installed in your system by running 
`pip install -e .` from the base directory of this repository.

In [ ]:
import sys
import os
import re
import glob
import math
import numpy as np
import pandas as pd
from datetime import datetime
import scipy.sparse as sp
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import multiprocessing
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch_geometric.nn import ChebConv
from tqdm import tqdm
from rwb_stgnn_functions import *
from rwb_dataloader_sunsmart import *
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Directorys
meta_directory = '/mnt/vstor/CSE_MSE_RXF131/staging/sdle/foundational_models/sunsmart/sunsmart_meta_grade.parquet'

# Dataloader
pvts = stDataloader(parent_directory = '/mnt/vstor/CSE_MSE_RXF131/staging/sdle/foundational_models/sunsmart/',
                        raw_parquet_path = '',
                        node_parquet_path ='',
                        channel_parquet_path = '',
                        adjacency_parquet_path = '', 
                        meta_parquet_path = meta_directory,
                        num_nodes = num_nodes, 
                        traintestsplit= traintestsplit,
                        splittype = splittype,
                        normalize = True,
                        model_output_directory='',
                        test_name = test_name)


Note that due to the complexity of this system, an integrated dataloader has been developed that operates off of config files. We'll load that in first.

We also define a set folder structure for the data, with optional functionality for data preprocessing. 

In [ ]:
# channel names
pvts.channel_names
# All dfs currently in channel_directory
pvts.channel_dataframes

# %% Data Preprocessing
# Reads in Raw Data
# Optional: splits tidy formatted data into separate csvs
# Add df manulipulation feature later
# Optional: writes to node_parquet path
#split_dfs = pvts.read_from_tabular(df_split_col='invt', write = False)

# %% Get Timespan
# Determines longest overlapping window
# From list of dfs
# Defaults self.node_dataframes
pvts.read_files_with_nodeID()
#timespan = pvts.get_longest_timespan(freq = '15min', tz = 'UTC',set_params = False)

#Set Timespan
pvts.set_date_range('2012-09-28 07:00:00', '2015-10-08 05:30:00', '15min', tz = 'UTC')

# Extracts columns from node df with multiple measurements, resizes based on self.time_index
# From list of node dfs
# Defaults self.node_dataframes
# Optional: write to self.channel_parquet_path
real_world_measurements = ['PVDCPOWER']

emp_model = ['p_mp']

sun_measurements = ['sun_angle']

weather_measurements = ['temp_air','dhi',
       'dni', 'ghi','wind_speed']

#Converts tabular data to matrix
_ = pvts.tabular_to_matrix(measurement_cols = real_world_measurements, write = False)
_ = pvts.tabular_to_matrix(measurement_cols = emp_model, write = False)
_ = pvts.tabular_to_matrix(measurement_cols = sun_measurements, write = False)
_ = pvts.tabular_to_matrix(measurement_cols = weather_measurements, write = False)

#Calculates adjacency matrix
_ = pvts.generate_distance_adjacency_matrix(latitude = 'lat', longitude='lon', epsilons = [0.0,0.2,0.4,0.6,0.8,1.0], write = False, weighted=False)
_ = pvts.generate_distance_adjacency_matrix(latitude = 'lat', longitude='lon', epsilons = [0.1,0.3,0.5,0.7,0.9], write = False)

_ = pvts.generate_datasetgrade_adjacency_matrix(grade_col = 'sdt_grade', min_grade_threshold = .5, max_diff_thresholds = [0.1,0.2,0.3,0.5,0.75,1.0], write = False)


# Extracts columns from node df with multiple measurements, resizes based on self.time_index, checks for NANs
# From list of node dfs
# Defaults self.node_dataframes
# Optional: Write to self.channel_parquet_path
_ = pvts.data_availibility(measurement_cols = real_world_measurements, write = False)

# %% Regularize Time interval for incommesurate indices 
# Reindexes timeseries based on dataloader timespan
# Only Linear Implemented 
# Timezone Aware
# _ = pvts.ts_reindex_with_interpolation('sun_angle',
#                                 start_freq = '15min',
#                                 method = 'linear'
#                                 )

# %% Conditional Masking / Data manipulation
# 
#
#
# N number of Conditional Statements to apply Masks:
#   ([(statement_1),(statement_2),...,(statement_n)], 
#           channel_if_true, channel_if_false, 
#           scalar_if_true, scalar_if_false,
#           new_channel_name
#       )

#   if (channel_x, conditional statement, channel_y, scalar value) 
#   AND 
#   (statement_2) ... AND (statement_n)
#   THEN
#   
#   [or to compare scalar value]
#   if (channel_z, conditional statement 2, None, scalar value)

good_data_filter = [([('ghi', np.greater, None, 50), ('PVDCPOWER', np.less, None, 5)], np.all, None, None, 1, 0, "ghi_fill_flag")]


mask = [([('ghi_fill_flag', np.equal, None, 1), ('PVDCPOWER', pd.isna, None, 1)], np.any, 'p_mp', 'PVDCPOWER', 1, 0, "cdcp_fill"),
        ([('sun_angle', np.less, None, .20), ('ghi_fill_flag', np.equal, None, 1), ('PVDCPOWER', pd.isna, None, 1)], np.any, None, None, 1, 0, "mask")]

mask = [([('sun_angle', np.less, None, .05)], np.any, None, None, 1, 0, "night_mask")]

_ = pvts.apply_conditions(good_data_filter)
_ = pvts.apply_conditions(mask, write = False)


These are the model inputs. This controls the architecture of the model.

In [ ]:
# Model Inputs
inputs = ['cdcp_fill']
outputs = ['cdcp_fill']
masks = ['mask']
adjacency = ['epsilon_0.7']

# Model Validation Options
traintestsplit = True
splittype = 'space'
splits = (.75,.20,.05)

# Model parameters
num_nodes = 90
num_channels = 1
#Number of Learned Filters per Temporal Conv Block
#First number is num 
channels = [[len(inputs), 9, 18], [18, 9, len(outputs)]]

#Temporal Convolutional Kernel Size
kernel_size = 4
#Temporal Deconvolutional Kernal Size (Second Layer)
kernel_size_de = 2

#Global Stride, Padding
stride = 2
padding = 1

#Chebchev Kernel (n-hops message passing)
K = 3

#Batch Size
batch_size = 1

# Training parameters
learning_rate = 0.0001
num_epochs = 5
num_layers = 2

#Temporal Window
window_size = 2880
shuffle = False

# Model Setup
test_name = 'autoencode_test'
model_num = 1

Now let's define where to save results and model objects:

In [ ]:

# Model Save Paths
model_name = 'model_' + str(model_num) + '.pt'
model_save_path = pvts.model_path + model_name
model_results_name = 'model_' + str(model_num) + '.csv'
model_results_path = pvts.model_results_path + model_results_name
pred_test_name = 'model_' + str(model_num) + '.parquet'
pred_test_save_path = pvts.pred_test_path + pred_test_name
pred_name = 'model_' + str(model_num) + '.parquet'
pred_save_path = pvts.pred_path + pred_name

Now, let's configure the dataloader:

In [ ]:
# Set Timespan
pvts.set_date_range('2012-09-28 07:00:00', '2015-10-08 05:30:00', '15min', tz = 'UTC')

# Set Model Inputs
pvts.set_window_size(window_size)
pvts.set_model_inputs(inputs, outputs, masks, adjacency)
pvts.set_train_test_splits(splits)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

You can optionally select specific nodes (colums) or time windows to evaluate your model.

In [ ]:
#Set Specific Train Cols (Optional)
pvts.train_cols = [0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 17, 18, 20, 21,
       22, 23, 24, 25, 26, 27, 30, 31, 32, 35, 36, 37, 38, 39, 40, 41, 42,
       44, 45, 46, 47, 49, 51, 52, 53, 55, 56, 59, 60, 61, 62, 63, 64, 65,
       67, 68, 69, 70, 74, 76, 78, 79, 81, 82, 83, 84, 85, 87, 88, 89]
pvts.test_cols = [14, 15, 19, 28, 29, 34, 43, 48, 50, 57, 58, 66, 71, 73, 75, 77, 80,
       86]
pvts.val_cols = [3, 16, 33, 54, 72]

Configure the tensor datasets, and the model iterable object.

In [ ]:
#Transform the Datafrom tables to matrices
train_data, test_data, val_data, full_data = pvts.data_to_model_input_transform()

train_iter = DataLoader(train_data, batch_size=int(batch_size),
                        shuffle=shuffle, num_workers=1)
test_iter = DataLoader(test_data, batch_size=int(batch_size),
                        shuffle=False, num_workers=1)
val_iter = DataLoader(val_data, batch_size=int(batch_size),
                        shuffle=False, num_workers=1)
full_iter = DataLoader(full_data, batch_size=int(batch_size),
                        shuffle=False, num_workers=1)

# %% Transform the adjacency matrix tables to matrices
train_w, train_edge_index, train_edge_weight, full_w, full_edge_index, full_edge_weight = pvts.weight_to_model_input_transform(datasplit = 'training')

# %% Initializes Model
model = STConvAE(device, num_nodes, channels, num_layers, 
                    kernel_size, K, window_size, kernel_size_de, stride, 
                    padding, normalization = 'sym', bias = True)
                    
model = model.to(device)

#Loss Functions
loss = nn.MSELoss()
min_test_loss = np.inf
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) 

#Model Loss
train_loss = []
test_loss = []

We can now begin the training loop

In [ ]:
for epoch in tqdm(range(1, num_epochs + 1), desc = 'Epoch', position = 0):

    epoch_train_loss = 0.0
    model.train()
    # i = 0
    
    for x, y, mask in tqdm(train_iter, desc = 'Batch', position = 0):
        
        optimizer.zero_grad()
        
        y = y.to(device)
        mask = mask.to(device)
        y_pred = model(x.to(device), 
                        train_edge_index.to(device), 
                        train_edge_weight.to(device)
                        ) 

        loss= torch.nanmean((torch.where(mask == False, 
                                (y_pred-y)**2, 
                                torch.tensor(float('nan'))
                                ))
                                )

        if not torch.isnan(loss):
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item() 
            train_loss.append(epoch_train_loss)

    epoch_test_loss = 0.0
    model.eval()

    if splittype == 'space':

        with torch.no_grad():
            
            for x, y, mask in tqdm(full_iter, desc = 'Batch', position = 0):

                y = y.to(device)
                mask = mask.to(device)
                y_pred = model(x.to(device),
                                    full_edge_index.to(device),
                                    full_edge_weight.to(device)
                                    ) 

                y = y[:,pvts.test_cols]
                mask = mask[:,pvts.test_cols]
                y_pred = y_pred[:,pvts.test_cols]
                
                loss = torch.nanmean((torch.where(mask == False,
                                        (y_pred-y)**2,
                                        torch.tensor(float('nan'))
                                        ))
                                        )

                if not torch.isnan(loss):
                    epoch_test_loss += loss.item()
                    test_loss.append(epoch_test_loss)

    else: 

        with torch.no_grad():

            for x, y, mask in tqdm(test_iter, desc = 'Batch', position = 0):

                y = y.to(device)
                mask = mask.to(device)
                y_pred = model(x.to(device),
                                    full_edge_index.to(device),
                                    full_edge_weight.to(device)
                                    ) 

                loss = torch.nanmean((torch.where(mask == False,
                                        (y_pred-y)**2,
                                        torch.tensor(float('nan'))
                                        ))
                                        )

                if not torch.isnan(loss):
                    epoch_test_loss += loss.item()
                    test_loss.append(epoch_test_loss)

    print(f'Epoch: {epoch} \n Training Loss: {epoch_train_loss / len(train_iter)} \n Evaulation Loss: {epoch_test_loss / len(test_iter)}')
    
    if min_test_loss > epoch_test_loss:
        min_test_loss = epoch_test_loss
        torch.save(model.state_dict(), model_save_path)

Evaluate the the best model:

In [ ]:
# Model Evaluation
# Intialize Model
eval_model = STConvAE(device, num_nodes, channels, num_layers, kernel_size, K, window_size, kernel_size_de, stride, padding, normalization = 'sym', bias = True).to(device)
eval_model.load_state_dict(torch.load(model_save_path))
eval_model = eval_model.to(device)
eval_loss = nn.MSELoss()
eval_model.eval()

if splittype == 'space':

    first_full = 1

    with torch.no_grad():
        
        for x, y, mask in tqdm(full_iter, desc = 'Batch', position = 0):

            torch.cuda.empty_cache()
            # get model predictions and compute loss
            y_pred = eval_model(x.to(device), 
                                    full_edge_index.to(device), 
                                    full_edge_weight.to(device)
                                    )
            
            if first_full == 1:
                y_complete = y[:,:,:,0].unsqueeze(-1)
                mask_complete = mask
                y_pred_complete = y_pred

            else:
                y_complete = torch.cat((y_complete, y[:,:,:,0].unsqueeze(-1)))
                mask_complete = torch.cat((mask_complete, mask))
                y_pred_complete = torch.cat((y_pred_complete, y_pred))

            first_full+=1

    raw = (y_complete).cpu().flatten(start_dim = 0, end_dim = 1).flatten(start_dim = 1, end_dim = 2).detach().numpy()
    raw = np.where(raw == 0, float(.1), raw)

    mask = (mask_complete).cpu().flatten(start_dim = 0, end_dim = 1).flatten(start_dim = 1, end_dim = 2).detach().numpy()
    pred = (y_pred_complete).cpu().flatten(start_dim = 0, end_dim = 1).flatten(start_dim = 1, end_dim = 2).detach().numpy()
    
    raw = raw[0:len(pvts.time_index)]
    mask = mask[0:len(pvts.time_index)]
    pred = pred[0:len(pvts.time_index)]

    raw_val = raw[:,pvts.val_cols]
    mask_val = mask[:,pvts.val_cols]
    pred_val = pred[:,pvts.val_cols]

elif splittype == 'time':

        first_val = 1

        for x, y, mask in tqdm(val_iter, desc = 'Batch', position = 0):

            torch.cuda.empty_cache()
            # get model predictions and compute loss
            y_pred = eval_model(x.to(device), 
                                    full_edge_index.to(device), 
                                    full_edge_weight.to(device)
                                    )
            
            if first_val == 1:
                y_complete = y[:,:,:,0].unsqueeze(-1)
                mask_complete = mask
                y_pred_complete = y_pred

            else:
                y_complete = torch.cat((y_complete, y[:,:,:,0].unsqueeze(-1)))
                mask_complete = torch.cat((mask_complete, mask))
                y_pred_complete = torch.cat((y_pred_complete, y_pred))

            first_val+=1

        raw_val = (y_complete).cpu().flatten(start_dim = 0, end_dim = 1).flatten(start_dim = 1, end_dim = 2).detach().numpy()


        mask_val = (mask_complete).cpu().flatten(start_dim = 0, end_dim = 1).flatten(start_dim = 1, end_dim = 2).detach().numpy()
        pred_val = (y_pred_complete).cpu().flatten(start_dim = 0, end_dim = 1).flatten(start_dim = 1, end_dim = 2).detach().numpy()

        first_full=1
        
        for x, y, mask in tqdm(full_iter, desc = 'Batch', position = 0):

            torch.cuda.empty_cache()

            # get model predictions and compute loss
            y_pred = eval_model(x.to(device), 
                                    full_edge_index.to(device), 
                                    full_edge_weight.to(device)
                                    )
            
            if first_full == 1:
                y_complete = y[:,:,:,0].unsqueeze(-1)
                mask_complete = mask
                y_pred_complete = y_pred

            else:
                y_complete = torch.cat((y_complete, y[:,:,:,0].unsqueeze(-1)))
                mask_complete = torch.cat((mask_complete, mask))
                y_pred_complete = torch.cat((y_pred_complete, y_pred))

            first_full+=1

        raw = (y_complete).cpu().flatten(start_dim = 0, end_dim = 1).flatten(start_dim = 1, end_dim = 2).detach().numpy()
        mask = (mask_complete).cpu().flatten(start_dim = 0, end_dim = 1).flatten(start_dim = 1, end_dim = 2).detach().numpy()
        pred = (y_pred_complete).cpu().flatten(start_dim = 0, end_dim = 1).flatten(start_dim = 1, end_dim = 2).detach().numpy()
        
        raw = raw[0:len(pvts.time_index)]
        mask = mask[0:len(pvts.time_index)]
        pred = pred[0:len(pvts.time_index)]

else:
    
    first_full = 1

    with torch.no_grad():
        
        for x, y, mask in tqdm(full_iter, desc = 'Batch', position = 0):

            torch.cuda.empty_cache()
            # get model predictions and compute loss
            y_pred = eval_model(x.to(device), 
                                    full_edge_index.to(device), 
                                    full_edge_weight.to(device)
                                    )
            
            if first_full == 1:
                y_complete = y[:,:,:,0].unsqueeze(-1)
                mask_complete = mask
                y_pred_complete = y_pred

            else:
                y_complete = torch.cat((y_complete, y[:,:,:,0].unsqueeze(-1)))
                mask_complete = torch.cat((mask_complete, mask))
                y_pred_complete = torch.cat((y_pred_complete, y_pred))

            first_full+=1

    raw = (y_complete).cpu().flatten(start_dim = 0, end_dim = 1).flatten(start_dim = 1, end_dim = 2).detach().numpy()
    raw = np.where(raw == 0, float(.1), raw)

    mask = (mask_complete).cpu().flatten(start_dim = 0, end_dim = 1).flatten(start_dim = 1, end_dim = 2).detach().numpy()
    pred = (y_pred_complete).cpu().flatten(start_dim = 0, end_dim = 1).flatten(start_dim = 1, end_dim = 2).detach().numpy()
    
    raw = raw[0:len(pvts.time_index)]
    mask = mask[0:len(pvts.time_index)]
    pred = pred[0:len(pvts.time_index)]

    raw_val = raw
    mask_val = mask
    pred_val = pred
# %% 
#Calculate Model Error
raw = np.where(raw == 0, float(.1), raw)
raw_val = np.where(raw_val == 0, float(.1), raw_val)

MAE_val = np.nanmean((np.where(mask_val == False, np.abs(pred_val - raw_val), np.nan)))
MAPE_val = np.nanmean((np.where(mask_val == False, np.abs((pred_val - raw_val) / raw_val), np.nan)))
MSE_val = np.nanmean((np.where(mask_val == False, (pred_val - raw_val)**2, np.nan)))
RMSE_val = ((np.nanmean((np.where(mask_val == False, (pred_val - raw_val)**2, np.nan))))**.5)

MAE = np.nanmean((np.where(mask == False, np.abs(pred - raw), np.nan)))
MAPE = np.nanmean((np.where(mask == False, np.abs((pred - raw) / raw), np.nan)))
MSE = np.nanmean((np.where(mask == False, (pred - raw)**2,  np.nan)))
RMSE = ((np.nanmean((np.where(mask == False, (pred - raw)**2,  np.nan))))**.5)

# raw.to_parquet()
# raw_val.to_parquet()
# mask.to_parquet()
# mask_val.to_parquet()
# pred_val.to_parquet()
# pred.to_parquet()


Compile model parameters and model results into a table:

In [ ]:
model_results_df = pd.DataFrame({
    'model_num' : model_num,
    'MAE_val': MAE_val,
    'MAPE_val': MAPE_val,
    'MSE_val': MSE_val,
    'RMSE_val': RMSE_val,
    'MAE': MAE,
    'MAPE': MAPE,
    'MSE': MSE,
    'RMSE': RMSE,
    'train_columns': [pvts.train_cols],
    'test_columns': [pvts.test_cols],
    'val_columns' : [pvts.val_cols],
    'train_indices': [pvts.train_windows],
    'test_indices': [pvts.test_windows],
    'val_indices' : [pvts.val_windows],
    'train_loss_curve': [train_loss],
    'test_loss_curve': [test_loss],
    'model_save_path' : model_save_path
})

#merged_df = pd.concat([model_results_df, input_df.reset_index()], axis=1)
model_results_df.to_csv(model_results_path)

pd.DataFrame(pred_val).to_parquet(pred_test_save_path)
pd.DataFrame(pred).to_parquet(pred_save_path)

Visualization of Results:

In [ ]:
sys_names = pvts.channel_dataframes[pvts.channel_indices['p_mp']].columns.values
valid_set = pd.DataFrame(pred_val)
valid_set.columns = sys_names[pvts.val_cols]
valid_set.columns = valid_set.columns.str.replace('_p_mp', '')
valid_set

In [ ]:
#valid_set = valid_set.iloc[:-101]
real_data = pvts.channel_dataframes[pvts.channel_indices['cdcp_fill']]
real_data.columns = real_data.columns.str.replace('_PVDCPOWER', '')
real_data

In [ ]:
real_data = real_data[valid_set.columns]

norms = np.nanmax(real_data, axis=0)
valid_set = (valid_set / 100) * norms 

emp_data = pvts.channel_dataframes[pvts.channel_indices['p_mp']]
emp_data.columns = emp_data.columns.str.replace('_p_mp', '')
emp_data = emp_data[valid_set.columns]
mask = pvts.channel_dataframes[pvts.channel_indices['mask']][pvts.val_cols].astype(bool)
mask.columns = valid_set.columns


Plotting Random Selections:

In [ ]:

# for start in range(1, 200):
for i, column_name in enumerate(real_data):
    for start in range(0,len(real_data), 25000):
        end = start + 300
        subset_real = real_data.values[start:end,i]
        subset_mask = mask.values[start:end,i]
        subset_emp = emp_data.values[start:end,i]
        subset_val = valid_set.values[start:end,i]
        plt.plot(np.where(subset_mask == False,subset_real,0), 'o', label='Real Data')
        plt.plot(subset_emp, label='Empirical Data')
        plt.plot(subset_val, label='Validation Set')

        plt.xlabel('Index')
        plt.ylabel('Values')
        plt.title(f'Comparison of Subsets {column_name}')
        plt.legend()
        plt.show()